# Introduction to AlgoSeek Datasets using the AlgoSeek-connector package

## 0. Set up logging

In [16]:
import logging

# create logger
logger = logging.getLogger('JupyterSamples')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.FileHandler('logs/daily_download.log')
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)


## 1) Overview

In [17]:
# import libraries
import algoseek_connector
import algoseek_connector.functions as fn
import pandas as pd
import os
from dotenv import load_dotenv
import clickhouse_driver

pd.set_option('display.max_columns',150)
pd.set_option('display.max_rows', 100)

load_dotenv()

jhost = os.getenv('julian_host')
juser = os.getenv('julian_user')
jpass = os.getenv('julian_password')

session = algoseek_connector.Session(
    jhost, juser, jpass, 9000
)
session.ping() # Verify that the session was instantiated corrected
resource = algoseek_connector.DataResource(session)

In [18]:
import clickhouse_driver
DATABASE_HOST = '54.164.83.119'
DATABASE_USER = "angel_mv_hill_gmail_com"
DATABASE_PASSWORD = "UbxaVAQ572A"
client = clickhouse_driver.Client(
    DATABASE_HOST,
    user=DATABASE_USER,
    password=DATABASE_PASSWORD,
    secure=False
)
client.execute('show databases')

[('USEquityMarketData',), ('USEquityReferenceData',)]

In [19]:
session = algoseek_connector.Session(DATABASE_HOST,DATABASE_USER,DATABASE_PASSWORD)
session.ping()
resource = algoseek_connector.DataResource(session)

In [5]:
import inspect
print(inspect.getmembers(resource))

[('__class__', <class 'algoseek_connector.resources.DataResource'>), ('__delattr__', <method-wrapper '__delattr__' of DataResource object at 0x000002334421FA30>), ('__dict__', {'session': <algoseek_connector.sessions.Session object at 0x0000023344215FD0>}), ('__dir__', <built-in method __dir__ of DataResource object at 0x000002334421FA30>), ('__doc__', None), ('__eq__', <method-wrapper '__eq__' of DataResource object at 0x000002334421FA30>), ('__format__', <built-in method __format__ of DataResource object at 0x000002334421FA30>), ('__ge__', <method-wrapper '__ge__' of DataResource object at 0x000002334421FA30>), ('__getattribute__', <method-wrapper '__getattribute__' of DataResource object at 0x000002334421FA30>), ('__gt__', <method-wrapper '__gt__' of DataResource object at 0x000002334421FA30>), ('__hash__', <method-wrapper '__hash__' of DataResource object at 0x000002334421FA30>), ('__init__', <bound method DataResource.__init__ of <algoseek_connector.resources.DataResource object a

In [8]:
for dataset in resource.datagroups.USEquityMarketData.datasets.all():
    print(dataset)

Dataset(USEquityMarketData.BasicAdjustedOHLCDaily)
Dataset(USEquityMarketData.BasicOHLCDaily)
Dataset(USEquityMarketData.PrimaryAdjustedOHLCDaily)
Dataset(USEquityMarketData.PrimaryOHLCDaily)
Dataset(USEquityMarketData.StandardAdjustedOHLCDaily)
Dataset(USEquityMarketData.StandardOHLCDaily)
Dataset(USEquityMarketData.TradeAndQuote)
Dataset(USEquityMarketData.TradeAndQuoteMinuteBar)
Dataset(USEquityMarketData.TradeAndQuoteMinuteBarExcludingTRF)
Dataset(USEquityMarketData.TradeOnly)
Dataset(USEquityMarketData.TradeOnlyAdjusted)
Dataset(USEquityMarketData.TradeOnlyAdjustedMinuteBar)
Dataset(USEquityMarketData.TradeOnlyAdjustedMinuteBarBBG)
Dataset(USEquityMarketData.TradeOnlyAdjustedMinuteBarExcludingTRF)
Dataset(USEquityMarketData.TradeOnlyMinuteBar)
Dataset(USEquityMarketData.TradeOnlyMinuteBarBBG)
Dataset(USEquityMarketData.TradeOnlyMinuteBarExcludingTRF)


In [6]:
for dataset in resource.datagroups.USEquityReferenceData.datasets.all():
    print(dataset)

Dataset(USEquityReferenceData.BasicAdjustments)
Dataset(USEquityReferenceData.CikLookupBase)
Dataset(USEquityReferenceData.CumulativePriceAdjustments)
Dataset(USEquityReferenceData.CumulativeVolumeAdjustments)
Dataset(USEquityReferenceData.DetailedAdjustments)
Dataset(USEquityReferenceData.FundamentalsFacts)
Dataset(USEquityReferenceData.LookupBase)
Dataset(USEquityReferenceData.MarketCalendar)
Dataset(USEquityReferenceData.MarketHolidays)
Dataset(USEquityReferenceData.SecMasterBase)


## 2) US Equity Reference Data

In [9]:
for dataset in resource.datagroups.USEquityReferenceData.datasets.all():
    print(dataset)

Dataset(USEquityReferenceData.BasicAdjustments)
Dataset(USEquityReferenceData.CikLookupBase)
Dataset(USEquityReferenceData.CumulativePriceAdjustments)
Dataset(USEquityReferenceData.CumulativeVolumeAdjustments)
Dataset(USEquityReferenceData.DetailedAdjustments)
Dataset(USEquityReferenceData.FundamentalsFacts)
Dataset(USEquityReferenceData.LookupBase)
Dataset(USEquityReferenceData.MarketCalendar)
Dataset(USEquityReferenceData.MarketHolidays)
Dataset(USEquityReferenceData.SecMasterBase)


### 2.1) Basic Adjustment Factors

In [16]:
basic_adj = resource.datagroups.USEquityReferenceData.datasets.BasicAdjustments
basic_adj.head().fetch().columns

Index(['SecId', 'Ticker', 'EffectiveDate', 'AdjustmentFactor',
       'AdjustmentReason', 'EventId'],
      dtype='object')

Basic Adjustment Factor Data Fields:

| Field            | Type (Format) | Description                                                                                      |
|------------------|---------------|--------------------------------------------------------------------------------------------------|
| SecId            | integer | Unique Security Identifier                                                                       |
| Ticker           | string | Ticker on the Effective Date                                                                     |
| EffectiveDate    | string (YYYYMMDD) | Date that the event becomes effective                                                            |
| AdjustmentFactor | decimal | The value of the adjustment factor for the event |
| AdjustmentReason | string | The reason for the Corporate Event. See subsection “Adjustment Reason” below for a list of types |
| EventId | integer | Unique Event ID under the AdjustmentReason, i.e., each pair of AdjustmentReason and EventId is unique |

In [16]:
basic_df = basic_adj.select(basic_adj.SecId,
                 basic_adj.Ticker,
                 basic_adj.EffectiveDate,
                 basic_adj.AdjustmentFactor,
                 basic_adj.AdjustmentReason,
                 basic_adj.EventId
).fetch()
basic_df.to_parquet('data/us_equity/reference/basic_adj.parquet')

### 2.2) CIK Lookup Base

In [87]:
cik_lookup = resource.datagroups.USEquityReferenceData.datasets.CikLookupBase
cik_lookup.head().fetch().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   SecId      10 non-null     int64 
 1   CIK        10 non-null     int64 
 2   StartDate  10 non-null     object
 3   EndDate    10 non-null     object
dtypes: int64(2), object(2)
memory usage: 448.0+ bytes


### 2.3) Cumulative Price Adjustments

In [92]:
cumulative_price_adjustments = resource.datagroups.USEquityReferenceData.datasets.CumulativePriceAdjustments
cumulative_price_adjustments.head().fetch().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SecId           10 non-null     int64  
 1   Ticker          10 non-null     object 
 2   EffectiveDate   10 non-null     object 
 3   ForwardFactor   10 non-null     float64
 4   BackwardFactor  10 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 528.0+ bytes


### 2.4) Cumulative Volume Adjustments

In [93]:
cumulative_volume_adjustments = resource.datagroups.USEquityReferenceData.datasets.CumulativeVolumeAdjustments
cumulative_volume_adjustments.head().fetch().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SecId           10 non-null     int64  
 1   Ticker          10 non-null     object 
 2   EffectiveDate   10 non-null     object 
 3   ForwardFactor   10 non-null     float64
 4   BackwardFactor  10 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 528.0+ bytes


### 2.5) Detailed Adjustment Factors

In [17]:
detailed_adj = resource.datagroups.USEquityReferenceData.datasets.DetailedAdjustments
detailed_adj.head().fetch().columns

Index(['SecId', 'Ticker', 'Name', 'ISIN', 'EffectiveDate', 'ReportDate',
       'Revision', 'AdjustmentFactor', 'AdjustmentReason', 'EventType',
       'EventId', 'DivPayrate', 'DivCurrency', 'DivPaymentType', 'Detail'],
      dtype='object')

Detailed Adjustment Factor data fields:

| Field            | Type (Format)               | Description                                                                                                                   |
|------------------|-----------------------------|-------------------------------------------------------------------------------------------------------------------------------|
| SecId            | integer                     | Unique security identifier                                                                                                    |
| Ticker           | string                      | Ticker on the Effective Date                                                                                                  |
| EffectiveDate    | string (YYYYMMDD)           | Date that the event becomes effective                                                                                         |
| Name             | string | Name of the company                                                                                                           |
| ReportDate       | string (YYYYMMDD[_N]        | Date and version (_N suffix) of the algoseek event report file                                                                |
| AdjustmentFactor | decimal                     | The value of the Adjustment factor for the event                                                                              |
| AdjustmentReason | string                      | The reason for the Corporate Event. See subsection "Adjustment Reason" below for a list of types                              |
| EventType        | string                      | Type of event. See section “Adjustment Reason Table” below for a list of types                                                |
| EventId          | integer                     | Unique Event ID under the AdjustmentReason, i.e., each pair of AdjustmentReason and EventId is unique                         |
| Detail           | string                      | Details of the event including Dividend Amount, Period (e.g., Interim), etc. See “Detail Field” below for further information |
| ISIN             | string                      | ISIN (International SecurNumber) Identifier|

In [18]:
det_df = detailed_adj.select(detailed_adj.SecId,
                    detailed_adj.Ticker,
                    detailed_adj.Name,
                    detailed_adj.ISIN,
                    detailed_adj.EffectiveDate,
                    detailed_adj.ReportDate,
                    detailed_adj.Revision,
                    detailed_adj.AdjustmentFactor,
                    detailed_adj.AdjustmentReason,
                    detailed_adj.EventType,
                    detailed_adj.EventId,
                    detailed_adj.DivPayrate,
                    detailed_adj.DivCurrency,
                    detailed_adj.DivPaymentType,
                    detailed_adj.Detail
                    ).fetch()
det_df.to_parquet('data/us_equity/reference/detailed_adj.parquet')

### 2.6) Fundamentals Facts

In [89]:
fundamental_facts = resource.datagroups.USEquityReferenceData.datasets.FundamentalsFacts
fundamental_facts.head().fetch().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CIK                    10 non-null     int64  
 1   CompanyName            10 non-null     object 
 2   Taxonomy               10 non-null     object 
 3   ReportingElement       10 non-null     object 
 4   ShortDescription       10 non-null     object 
 5   LongDescription        10 non-null     object 
 6   ReportingElementValue  10 non-null     float64
 7   Units                  10 non-null     object 
 8   FilingStartDate        10 non-null     object 
 9   FilingEndDate          10 non-null     object 
 10  FilingYear             10 non-null     int64  
 11  FilingPeriod           10 non-null     object 
 12  FilingForm             10 non-null     object 
 13  FilingDate             10 non-null     object 
 14  FilingFrame            10 non-null     object 
 15  Accession

### 2.7) Lookup Base

In [11]:
lookup_base = resource.datagroups.USEquityReferenceData.datasets.LookupBase
lookup_base.head().fetch().columns

Index(['SecId', 'Ticker', 'StartDate', 'EndDate'], dtype='object')

Lookup Base Fields

In [19]:
look_df = lookup_base.select(lookup_base.SecId,
                   lookup_base.Ticker,
                   lookup_base.StartDate,
                   lookup_base.EndDate
).fetch()
look_df.to_parquet('data/us_equity/reference/lookup_base.parquet')

### 2.8) Market Calendar

In [90]:
market_calendar = resource.datagroups.USEquityReferenceData.datasets.MarketCalendar
market_calendar.head().fetch().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CalendarDate        10 non-null     object
 1   IsTradingDate       10 non-null     int64 
 2   PreviousTradingDay  10 non-null     object
 3   NextTradingDay      10 non-null     object
dtypes: int64(1), object(3)
memory usage: 448.0+ bytes


### 2.9) Market Holidays

In [91]:
market_holidays = resource.datagroups.USEquityReferenceData.datasets.MarketHolidays
market_holidays.head().fetch().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    10 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes


### 2.10) SEC Master Base

In [12]:
sec_master_base = resource.datagroups.USEquityReferenceData.datasets.SecMasterBase
sec_master_base.head().fetch().columns

Index(['SecId', 'ListStatus', 'SecurityDescription', 'Sic', 'Sector',
       'Industry', 'SEDOL', 'Ticker', 'TickerStartToEndDate', 'Name',
       'NameStartToEndDate', 'ISIN', 'ISINStartToEndDate', 'USIdentifier',
       'USIdentifierStartToEndDate', 'PrimaryExchange',
       'PrimaryExchangeStartToEndDate', 'FIGI'],
      dtype='object')

SEC Master Base Datafields

In [ ]:
sec_df = sec_master_base.select(sec_master_base.SecId,
                       sec_master_base.ListStatus,
                       sec_master_base.SecurityDescription,
                       sec_master_base.Sic,
                       sec_master_base.Sector,
                       sec_master_base.Industry,
                       sec_master_base.SEDOL,
                       sec_master_base.Ticker,
                       sec_master_base.TickerStartToEndDate,
                       sec_master_base.Name,
                       sec_master_base.NameStartToEndDate,
                       sec_master_base.ISIN,
                       sec_master_base.ISINStartToEndDate,
                       sec_master_base.USIdentifier,
                       sec_master_base.USIdentifierStartToEndDate,
                       sec_master_base.PrimaryExchange,
                       sec_master_base.PrimaryExchangeStartToEndDate,
                       sec_master_base.FIGI
).fetch()
sec_df.to_parquet('data/us_equity/reference/sec_master.parquet')

## 3) US Equity Market Data

These cells deal with downloading market data for US Equities. Run whichever cells download the data that you want then skip the rest. Make sure you go to the end to download the reference data.

In [17]:
# List all available datasets
for dataset in resource.datagroups.USEquityMarketData.datasets.all():
    print(dataset.name)

BasicAdjustedOHLCDaily
BasicOHLCDaily
PrimaryAdjustedOHLCDaily
PrimaryOHLCDaily
StandardAdjustedOHLCDaily
StandardOHLCDaily
TradeAndQuote
TradeAndQuoteMinuteBar
TradeAndQuoteMinuteBarExcludingTRF
TradeOnly
TradeOnlyAdjusted
TradeOnlyAdjustedMinuteBar
TradeOnlyAdjustedMinuteBarBBG
TradeOnlyAdjustedMinuteBarExcludingTRF
TradeOnlyMinuteBar
TradeOnlyMinuteBarBBG
TradeOnlyMinuteBarExcludingTRF


### 3.1) Basic OHLC Daily

### 3.2) Basic Adjusted OHLC Daily

### 3.3) Primary OHLC Daily

In [7]:
primary_adj = resource.datagroups.USEquityMarketData.datasets.PrimaryAdjustedOHLCDaily
primary_adj.head().fetch().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype                     
---  ------                   --------------  -----                     
 0   TradeDate                10 non-null     object                    
 1   SecId                    10 non-null     int64                     
 2   Ticker                   10 non-null     object                    
 3   Name                     10 non-null     object                    
 4   PrimaryExchange          10 non-null     object                    
 5   ISIN                     10 non-null     object                    
 6   OpenTime                 10 non-null     datetime64[ns, US/Eastern]
 7   OpenPrice                10 non-null     float64                   
 8   OpenSize                 10 non-null     int64                     
 9   HighTime                 10 non-null     datetime64[ns, US/Eastern]
 10  HighPrice        

In [12]:
look = pd.read_parquet('data/us_equity/reference/lookup.parquet')
tickers = look['Ticker'].unique()

In [43]:
tickers

array(['SLF', 'ASTSF', 'PGLA', ..., 'IILGV', 'WRKw', 'RACEw'],
      dtype=object)

In [23]:
%%time
aapl = primary_adj.select(
   primary_adj.TradeDate, primary_adj.SecId, primary_adj.Ticker, primary_adj.Name, primary_adj.PrimaryExchange, primary_adj.ISIN,
       primary_adj.OpenTime, primary_adj.OpenPrice, primary_adj.OpenSize, primary_adj.HighTime, primary_adj.HighPrice, primary_adj.LowTime,
       primary_adj.LowPrice, primary_adj.CloseTime, primary_adj.ClosePrice, primary_adj.CloseSize,
       primary_adj.ListedMarketHoursVolume, primary_adj.ListedMarketHoursTrades,
       primary_adj.ListedTotalVolume, primary_adj.ListedTotalTrades, primary_adj.FinraMarketHoursVolume,
       primary_adj.FinraMarketHoursTrades, primary_adj.FinraTotalVolume, primary_adj.FinraTotalTrades,
       primary_adj.MarketVWAP, primary_adj.DailyVWAP, primary_adj.PriceAdjFactor, primary_adj.VolumeAdjFactor).filter((primary_adj.Ticker == 'AAPL')&
    (primary_adj.TradeDate > '2022-05-27')).fetch()
aapl.to_csv('C:/Users/julia/Documents/aapl_test.csv')

CPU times: total: 62.5 ms
Wall time: 2.82 s


In [22]:
aapl

,TradeDate,SecId,Ticker,Name,PrimaryExchange,ISIN,OpenTime,OpenPrice,OpenSize,HighTime,HighPrice,LowTime,LowPrice,CloseTime,ClosePrice,CloseSize,ListedMarketHoursVolume,ListedMarketHoursTrades,ListedTotalVolume,ListedTotalTrades,FinraMarketHoursVolume,FinraMarketHoursTrades,FinraTotalVolume,FinraTotalTrades,MarketVWAP,DailyVWAP,PriceAdjFactor,VolumeAdjFactor
0,2022-08-25,33449,AAPL,Apple Inc,NASDAQ,US0378331005,2022-08-25 09:30:00.590000-04:00,168.4006,442582,2022-08-25 15:59:59.649000-04:00,169.8582,2022-08-25 12:49:02.401000-04:00,168.0712,2022-08-25 16:00:00.846000-04:00,169.7484,4912215,28142766,271154,28724961,283180,21133194,176980,22493875,182223,169.059555,169.069538,0.998344,1.0
1,2022-10-07,33449,AAPL,Apple Inc,NASDAQ,US0378331005,2022-10-07 09:30:01.787000-04:00,142.3439,474471,2022-10-07 09:40:19.583000-04:00,142.8630,2022-10-07 15:34:10.780000-04:00,139.2191,2022-10-07 16:00:01.334000-04:00,139.8580,9580196,48032189,384173,49937857,410395,33384392,248347,35987936,256067,140.536870,140.556837,0.998344,1.0
2,2022-07-14,33449,AAPL,Apple Inc,NASDAQ,US0378331005,2022-07-14 09:30:00.446000-04:00,143.5920,305803,2022-07-14 15:24:16.294000-04:00,148.4971,2022-07-14 09:37:16.742000-04:00,142.8144,2022-07-14 16:00:00.142000-04:00,148.0185,4571567,38880343,380540,39736550,394507,36581432,215418,38404295,220446,146.134259,146.144229,0.996959,1.0
3,2022-08-26,33449,AAPL,Apple Inc,NASDAQ,US0378331005,2022-08-26 09:30:00.541000-04:00,170.2077,354674,2022-08-26 10:10:05.986000-04:00,170.7667,2022-08-26 15:44:23.506000-04:00,163.2891,2022-08-26 16:00:00.422000-04:00,163.3490,10991712,45485365,429875,46864231,452709,29431347,260800,32184528,269308,165.784987,165.725086,0.998344,1.0
4,2022-05-31,33449,AAPL,Apple Inc,NASDAQ,US0378331005,2022-05-31 09:30:01.601000-04:00,148.6964,1093978,2022-05-31 13:09:39.536000-04:00,150.2019,2022-05-31 10:14:23.957000-04:00,146.4034,2022-05-31 16:00:01.081000-04:00,148.3874,16457292,54446089,411086,55719291,433867,37959772,265447,48000037,272791,148.666535,148.646596,0.996959,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2022-11-17,33449,AAPL,Apple Inc,NASDAQ,US0378331005,2022-11-17 09:30:01.421000-05:00,146.5100,678568,2022-11-17 13:20:59.080000-05:00,151.4800,2022-11-17 09:30:01.331000-05:00,146.1500,2022-11-17 16:00:00.901000-05:00,150.7200,5522393,35030676,316096,36165067,335362,39485685,222082,44308980,228361,149.700000,149.710000,1.000000,1.0
116,2022-08-23,33449,AAPL,Apple Inc,NASDAQ,US0378331005,2022-08-23 09:30:00.104000-04:00,166.8033,561957,2022-08-23 10:11:13.588000-04:00,168.4306,2022-08-23 11:03:24.823000-04:00,166.3740,2022-08-23 16:00:01.600000-04:00,166.9530,3381056,28757255,292067,29481919,305454,21769061,183378,24665812,189239,167.372354,167.342404,0.998344,1.0
117,2022-07-12,33449,AAPL,Apple Inc,NASDAQ,US0378331005,2022-07-12 09:30:01.440000-04:00,145.3467,539225,2022-07-12 12:47:31.045000-04:00,147.9986,2022-07-12 15:38:24.586000-04:00,144.6089,2022-07-12 16:00:01.559000-04:00,145.4164,6492048,41000169,357370,41789829,369418,34369151,205869,35804195,210117,146.383499,146.343621,0.996959,1.0
118,2022-08-24,33449,AAPL,Apple Inc,NASDAQ,US0378331005,2022-08-24 09:30:01.694000-04:00,166.7833,599937,2022-08-24 15:42:37.603000-04:00,167.8316,2022-08-24 11:04:33.500000-04:00,165.9747,2022-08-24 16:00:00.838000-04:00,167.2526,4101502,28833617,277694,29807777,292184,23107357,180148,24152268,184792,167.022933,167.032917,0.998344,1.0


In [20]:
%%time
aapl1 = primary_adj.select(
   primary_adj.TradeDate, primary_adj.SecId, primary_adj.Ticker, primary_adj.Name, primary_adj.PrimaryExchange, primary_adj.ISIN,
       primary_adj.OpenTime, primary_adj.OpenPrice, primary_adj.OpenSize, primary_adj.HighTime, primary_adj.HighPrice, primary_adj.LowTime,
       primary_adj.LowPrice, primary_adj.CloseTime, primary_adj.ClosePrice, primary_adj.CloseSize,
       primary_adj.ListedMarketHoursVolume, primary_adj.ListedMarketHoursTrades,
       primary_adj.ListedTotalVolume, primary_adj.ListedTotalTrades, primary_adj.FinraMarketHoursVolume,
       primary_adj.FinraMarketHoursTrades, primary_adj.FinraTotalVolume, primary_adj.FinraTotalTrades,
       primary_adj.MarketVWAP, primary_adj.DailyVWAP, primary_adj.PriceAdjFactor, primary_adj.VolumeAdjFactor).filter((primary_adj.Ticker == 'AAPL')&
    (primary_adj.TradeDate > '2020-01-01')).fetch()
aapl1

ExecutionError: 1.00 GiB, current bytes: 1.01 GiB: While executing MergeTreeThread. 

### 3.4) Trade And Quote Minute Bar

In [35]:
taq_min = resource.datagroups.USEquityMarketData.datasets.TradeAndQuoteMinuteBar
taq_min.head().fetch().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 62 columns):
 #   Column                       Non-Null Count  Dtype                     
---  ------                       --------------  -----                     
 0   TradeDate                    10 non-null     object                    
 1   BarDateTime                  10 non-null     datetime64[ns, US/Eastern]
 2   Ticker                       10 non-null     object                    
 3   SecId                        10 non-null     int64                     
 4   OpenBarTimeOffset            10 non-null     object                    
 5   OpenBidPrice                 10 non-null     object                    
 6   OpenBidSize                  10 non-null     int64                     
 7   OpenAskPrice                 10 non-null     object                    
 8   OpenAskSize                  10 non-null     int64                     
 9   FirstTradeTimeOffset         10 non-null     o

In [74]:
taq_min.head().fetch().columns

Index(['TradeDate', 'BarDateTime', 'Ticker', 'SecId', 'OpenBarTimeOffset',
       'OpenBidPrice', 'OpenBidSize', 'OpenAskPrice', 'OpenAskSize',
       'FirstTradeTimeOffset', 'FirstTradePrice', 'FirstTradeSize',
       'HighBidTimeOffset', 'HighBidPrice', 'HighBidSize', 'HighAskTimeOffset',
       'HighAskPrice', 'HighAskSize', 'HighTradeTimeOffset', 'HighTradePrice',
       'HighTradeSize', 'LowBidTimeOffset', 'LowBidPrice', 'LowBidSize',
       'LowAskTimeOffset', 'LowAskPrice', 'LowAskSize', 'LowTradeTimeOffset',
       'LowTradePrice', 'LowTradeSize', 'CloseBarTimeOffset', 'CloseBidPrice',
       'CloseBidSize', 'CloseAskPrice', 'CloseAskSize', 'LastTradeTimeOffset',
       'LastTradePrice', 'LastTradeSize', 'MinSpread', 'MaxSpread',
       'CancelSize', 'VolumeWeightPrice', 'NBBOQuoteCount', 'TradeAtBid',
       'TradeAtBidMid', 'TradeAtMid', 'TradeAtMidAsk', 'TradeAtAsk',
       'TradeAtCrossOrLocked', 'Volume', 'TotalTrades', 'FinraVolume',
       'FinraVolumeWeightPrice', 'Upti

In [81]:
import inspect



Dataset(USEquityMarketData.TradeAndQuoteMinuteBar)


In [85]:
data = taq_min.select('TradeDate', 'BarDateTime', 'Ticker', 'SecId', 'OpenBarTimeOffset',
       'OpenBidPrice', 'OpenBidSize', 'OpenAskPrice', 'OpenAskSize',
       'FirstTradeTimeOffset', 'FirstTradePrice', 'FirstTradeSize',
       'HighBidTimeOffset', 'HighBidPrice', 'HighBidSize', 'HighAskTimeOffset',
       'HighAskPrice', 'HighAskSize', 'HighTradeTimeOffset', 'HighTradePrice',
       'HighTradeSize', 'LowBidTimeOffset', 'LowBidPrice', 'LowBidSize',
       'LowAskTimeOffset', 'LowAskPrice', 'LowAskSize', 'LowTradeTimeOffset',
       'LowTradePrice', 'LowTradeSize', 'CloseBarTimeOffset', 'CloseBidPrice',
       'CloseBidSize', 'CloseAskPrice', 'CloseAskSize', 'LastTradeTimeOffset',
       'LastTradePrice', 'LastTradeSize', 'MinSpread', 'MaxSpread',
       'CancelSize', 'VolumeWeightPrice', 'NBBOQuoteCount', 'TradeAtBid',
       'TradeAtBidMid', 'TradeAtMid', 'TradeAtMidAsk', 'TradeAtAsk',
       'TradeAtCrossOrLocked', 'Volume', 'TotalTrades', 'FinraVolume',
       'FinraVolumeWeightPrice', 'UptickVolume', 'DowntickVolume',
       'RepeatUptickVolume', 'RepeatDowntickVolume', 'UnknownTickVolume',
       'TradeToMidVolWeight', 'TradeToMidVolWeightRelative', 'TimeWeightBid',
       'TimeWeightAsk').filter((taq_min.Ticker == 'AAPL') & (taq_min.TradeDate == '2023-01-11')).head().fetch()
data

,TradeDate,BarDateTime,Ticker,SecId,OpenBarTimeOffset,OpenBidPrice,OpenBidSize,OpenAskPrice,OpenAskSize,FirstTradeTimeOffset,FirstTradePrice,FirstTradeSize,HighBidTimeOffset,HighBidPrice,HighBidSize,HighAskTimeOffset,HighAskPrice,HighAskSize,HighTradeTimeOffset,HighTradePrice,HighTradeSize,LowBidTimeOffset,LowBidPrice,LowBidSize,LowAskTimeOffset,LowAskPrice,LowAskSize,LowTradeTimeOffset,LowTradePrice,LowTradeSize,CloseBarTimeOffset,CloseBidPrice,CloseBidSize,CloseAskPrice,CloseAskSize,LastTradeTimeOffset,LastTradePrice,LastTradeSize,MinSpread,MaxSpread,CancelSize,VolumeWeightPrice,NBBOQuoteCount,TradeAtBid,TradeAtBidMid,TradeAtMid,TradeAtMidAsk,TradeAtAsk,TradeAtCrossOrLocked,Volume,TotalTrades,FinraVolume,FinraVolumeWeightPrice,UptickVolume,DowntickVolume,RepeatUptickVolume,RepeatDowntickVolume,UnknownTickVolume,TradeToMidVolWeight,TradeToMidVolWeightRelative,TimeWeightBid,TimeWeightAsk
0,2023-01-11,2023-01-11 04:00:00-05:00,AAPL,33449,0,0,0,0,0,0.00629341,130.95,100,0.042018673,131,100,0.048586585,131,200,0.040824784,131.03,585,0.004351008,130.68,400,51.4154176,130.8,100,52.805085286,130.74,34,59.999999999,130.73,200,130.8,100,52.805085286,130.74,34,0.01,0.27,0,130.93145,213,100,179,0,654,2302,601,3836,64,0,0,823,837,601,660,915,8.68131,0.36743,130.7432,130.89634
1,2023-01-11,2023-01-11 04:01:00-05:00,AAPL,33449,0,130.73,200,130.8,100,0.531453135,130.8,1,58.387718711,130.9,100,0.537108847,131,100,11.107681383,130.98,10,0,130.73,200,0,130.8,100,2.032180098,130.74,1,59.999999999,130.89,100,130.98,500,58.727162342,130.96,47,0.04,0.27,0,130.8448,96,531,118,0,479,862,0,1990,46,0,0,486,261,597,646,0,-0.53618,0.13867,130.84438,130.9726
2,2023-01-11,2023-01-11 04:02:00-05:00,AAPL,33449,0,130.89,100,130.98,500,3.448826848,130.98,6,23.166551595,131.08,100,23.062428755,131.25,100,22.99835422,131.08,50,0,130.89,100,0,130.98,500,11.123706118,130.91,50,59.999999999,130.9,200,131.07,500,39.458154726,131.07,50,0.03,0.36,0,130.98954,132,173,149,0,1431,275,0,2028,44,0,0,525,287,1116,100,0,2.8607,0.22311,130.89618,131.05252
3,2023-01-11,2023-01-11 04:03:00-05:00,AAPL,33449,0,130.9,200,131.07,500,13.953163979,130.96,1,47.003763503,131.01,100,14.636022501,131.25,100,47.003717758,131.15,198,0,130.9,200,8.860117216,131.05,100,13.953163979,130.96,1,59.999999999,130.97,500,131.17,100,55.371504855,131.12,19,0.14,0.35,0,131.13142,126,0,11,0,281,344,0,636,14,0,0,444,64,110,18,0,7.6934,0.43196,130.93642,131.12393
4,2023-01-11,2023-01-11 04:04:00-05:00,AAPL,33449,0,130.97,500,131.17,100,0.018859468,131.14,20,7.75359141,131.14,100,16.792894775,131.24,400,7.12841056,131.16,10,57.12975622,130.95,100,7.756229676,131.14,1000,37.753142294,131.03,12,59.999999999,131,400,131.17,400,59.297191676,131.13,1,0.02,0.23,0,131.10701,70,13,311,0,128,10,0,462,19,0,0,79,177,69,137,0,-1.27381,-0.05933,131.03144,131.18123
5,2023-01-11,2023-01-11 04:05:00-05:00,AAPL,33449,0,131,400,131.17,400,17.711616556,131.13,20,28.27697654,131.16,100,1.549748469,131.25,100,49.199206039,131.2,238,1.154017404,130.95,100,41.704929402,131.13,800,59.773591992,131.12,1,59.999999999,131.07,600,131.2,500,59.773591992,131.12,1,0.06,0.3,0,131.17528,90,11,22,0,780,598,0,1411,22,0,0,624,28,534,225,0,4.79341,0.43102,131.05584,131.20316
6,2023-01-11,2023-01-11 04:06:00-05:00,AAPL,33449,0,131.07,600,131.2,500,0.894757427,131.16,2,41.336780818,131.15,100,7.195659787,131.25,100,11.312931734,131.24,11,0,131.07,600,48.760688158,131.18,100,3.449050358,131.12,1,59.999999999,131.13,100,131.24,400,57.343398014,131.15,1,0.05,0.18,0,131.17377,44,80,47,8,107,10,0,252,15,0,0,35,125,92,0,0,-0.52778,-0.10993,131.10442,131.2397
7,2023-01-11,2023-01-11 04:07:00-05:00,AAPL,33449,0,131.13,100,131.24,400,1.672649915,131.15,1,7.095699442,131.14,100,7.095677551,131.25,200,5.45639819,131.19,18,0,131.13,100,7.09525975,131.16,400,1.672649915,131.15,1,59.999999999,131.13,100,131.19,400,48.518972246,131.18,118,0.03,0.12,0,131.16568,24,0,401,0,143,0,0,544,6,0,0,24,401,1,118,0,-1.73162,-0.10627,131.13,131.21154
8,2023-01-11,2023-0

In [86]:
pa.Table.from_pandas(data)

pyarrow.Table
TradeDate: date32[day]
BarDateTime: timestamp[ns, tz=US/Eastern]
Ticker: string
SecId: int64
OpenBarTimeOffset: decimal128(1, 0)
OpenBidPrice: decimal128(5, 2)
OpenBidSize: int64
OpenAskPrice: decimal128(5, 2)
OpenAskSize: int64
FirstTradeTimeOffset: decimal128(11, 9)
FirstTradePrice: decimal128(5, 2)
FirstTradeSize: int64
HighBidTimeOffset: decimal128(11, 9)
HighBidPrice: decimal128(5, 2)
HighBidSize: int64
HighAskTimeOffset: decimal128(11, 9)
HighAskPrice: decimal128(5, 2)
HighAskSize: int64
HighTradeTimeOffset: decimal128(11, 9)
HighTradePrice: decimal128(5, 2)
HighTradeSize: int64
LowBidTimeOffset: decimal128(11, 9)
LowBidPrice: decimal128(5, 2)
LowBidSize: int64
LowAskTimeOffset: decimal128(11, 9)
LowAskPrice: decimal128(5, 2)
LowAskSize: int64
LowTradeTimeOffset: decimal128(11, 9)
LowTradePrice: decimal128(5, 2)
LowTradeSize: int64
CloseBarTimeOffset: decimal128(11, 9)
CloseBidPrice: decimal128(5, 2)
CloseBidSize: int64
CloseAskPrice: decimal128(5, 2)
CloseAskSize: 

In [84]:
tm = ds.dataset('data/us_equity/dataset'
)
tm.schema

KeyboardInterrupt: 

## 4. Concurrent Downloads Example

### 4.1. Daily Datasets - Primary Adjusted Daily

threading

In [ ]:
import threading, queue
import os
import requests
import pandas as pd
import time
from pandas.core.frame import DataFrame

num_workers = 3
records = 50

results = []
failed_requests = []
Ids = set()

def worker(worker_num:int, q:queue) -> None:
    with requests.Session() as session:
        while True:
            Ids.add(f'Worker: {worker_num}, PID: {os.getpid()}, TID: {threading.get_ident()}')
            category, id = q.get()
            endpoint = f'/inventory/{category}?id={id}'
            print(f'WORKER {worker_num}: API request for cat: {category}, id: {id} started ...')
            response = session.get(url=base_url+endpoint)
            if response.ok:
                results.append(response.json())
            else:
                failed_requests.append((category, id))
            q.task_done()

def main() -> DataFrame:
    # Create queue and add items
    q = queue.Queue()
    for category in ('materials', 'products'):
        for id in range(records):
            q.put((category,id))

    # turn-on the worker thread(s)
    # daemon: runs without blocking the main program from exiting
    for i in range(num_workers):
        threading.Thread(target=worker, args=(i, q), daemon=True).start()

    # block until all tasks are done
    q.join()

    # make dataframe of results
    return pd.DataFrame(results)


print('THREADING')
start_time = time.time()
df = main()
print(f'\nDataframe ({len(failed_requests)} failed requests, {len(results)} successful requests)\n {df.head()}')
print("\n--- %s seconds ---" % (time.time() - start_time))
print(list(Ids))

Asyncio

In [20]:
import asyncio
import aiohttp
import threading, os
import time
import pandas as pd
from pandas.core.frame import DataFrame
import clickhouse_driver
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

num_tasks = 5
records = 50
years=[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,2022]

results = []
failed_requests = []
Ids = set()

async def task(task_id, work_queue) -> None:
    while not work_queue.empty():
        client = clickhouse_driver.Client(
        DATABASE_HOST,
        user=DATABASE_USER,
        password=DATABASE_PASSWORD,
        secure=False
        )
        session = algoseek_connector.Session(DATABASE_HOST,DATABASE_USER,DATABASE_PASSWORD)
        resource = algoseek_connector.DataResource(session)

        Ids.add(f'Task: {task_id}, PID: {os.getpid()}, TID: {threading.get_ident()}')
        company, year = await work_queue.get()
        #endpoint = f'/inventory/{category}?id={id}'
        resource = algoseek_connector.DataResource(session)
        primary_adj = resource.datagroups.USEquityMarketData.datasets.PrimaryAdjustedOHLCDaily
        try:
            data = primary_adj.select(primary_adj.TradeDate, primary_adj.SecId, primary_adj.Ticker, primary_adj.Name, primary_adj.PrimaryExchange, primary_adj.ISIN,primary_adj.OpenTime, primary_adj.OpenPrice, primary_adj.OpenSize, primary_adj.HighTime, primary_adj.HighPrice, primary_adj.LowTime,primary_adj.LowPrice, primary_adj.CloseTime, primary_adj.ClosePrice, primary_adj.CloseSize,primary_adj.ListedMarketHoursVolume, primary_adj.ListedMarketHoursTrades,primary_adj.ListedTotalVolume, primary_adj.ListedTotalTrades, primary_adj.FinraMarketHoursVolume,primary_adj.FinraMarketHoursTrades, primary_adj.FinraTotalVolume, primary_adj.FinraTotalTrades,primary_adj.MarketVWAP, primary_adj.DailyVWAP, primary_adj.PriceAdjFactor, primary_adj.VolumeAdjFactor).filter((primary_adj.Ticker == f"{company}")&(primary_adj.TradeDate > f"{str(year)}-01-01")&(primary_adj.TradeDate <= f"{str(year+1)}-01-01")).fetch()
            data["TradeDate"] = pd.to_datetime(data["TradeDate"])
            data["Year"] = data["TradeDate"].dt.year
            data_arrow = pa.Table.from_pandas(data)

            pq.write_to_dataset(data_arrow, root_path="data/us_equity/example_daily/",partition_cols=["Year","SecId"], use_legacy_dataset=False)
            # data.to_parquet(f"{company}_{str(year)}.parquet")
            logger.info(f'TASK {task_id}: API request for company: {company}, year: {year}')
            work_queue.task_done()
        except:
            logger.error(f"{company}_{year} failed")

async def main() -> None:
    # Create the queue of work
    q = asyncio.Queue()

    # Put some work in the queue
    for company in tickers:
        for year in years:
            await q.put((company,year))
    # Schedule tasks concurrently
    await asyncio.gather(
        *[asyncio.create_task(task(task_id, q)) for task_id in range(num_tasks)]
    )

    # make dataframe of results
    # return pd.DataFrame(results)

def download_data():

    print('ASYNCIO')
    start_time = time.time()
    asyncio.get_event_loop().run_until_complete(main())
    print(f'\nDataframe ({len(failed_requests)} failed requests, {len(results)} successful requests)\n {df.head()}')
    print("\n--- %s seconds ---" % (time.time() - start_time))
    print(list(Ids))

In [21]:
download_data()

ASYNCIO


C:\Users\julia\AppData\Roaming\Python\Python39\site-packages\clickhouse_driver\block.py:23: RuntimeWarning: coroutine 'main' was never awaited
  field_num = read_varint(buf)
Error on 54.164.83.119:9000 ping: timed out
Connection was closed, reconnecting.


ExecutionError: result_bytes = 107374419455/107374182400. Interval will end at 2023-01-17 00:00:00. Name of quota template: `qD_angel_mv_hill_gmail_com`. 

In [5]:
asyncio.get_event_loop().stop()

In [ ]:
print('ASYNCIO')
start_time = time.time()
df = asyncio.get_event_loop().run_until_complete(main())
print(f'\nDataframe ({len(failed_requests)} failed requests, {len(results)} successful requests)\n {df.head()}')
print("\n--- %s seconds ---" % (time.time() - start_time))
print(list(Ids))

ASYNCIO


In [40]:
dataset = pq.ParquetDataset("example_daily/", use_legacy_dataset=False)
dataset.schema

TradeDate: timestamp[us]
Ticker: string
Name: string
PrimaryExchange: string
ISIN: string
OpenTime: timestamp[us, tz=US/Eastern]
OpenPrice: double
OpenSize: int64
HighTime: timestamp[us, tz=US/Eastern]
HighPrice: double
LowTime: timestamp[us, tz=US/Eastern]
LowPrice: double
CloseTime: timestamp[us, tz=US/Eastern]
ClosePrice: double
CloseSize: int64
ListedMarketHoursVolume: int64
ListedMarketHoursTrades: int64
ListedTotalVolume: int64
ListedTotalTrades: int64
FinraMarketHoursVolume: int64
FinraMarketHoursTrades: int64
FinraTotalVolume: int64
FinraTotalTrades: int64
MarketVWAP: double
DailyVWAP: double
PriceAdjFactor: double
VolumeAdjFactor: double
SecId: dictionary<values=int32, indices=int32, ordered=0>
Year: dictionary<values=int32, indices=int32, ordered=0>
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 3945

In [ ]:
# Test using

In [9]:

data = primary_adj.select(primary_adj.TradeDate, primary_adj.SecId, primary_adj.Ticker, primary_adj.Name, primary_adj.PrimaryExchange, primary_adj.ISIN,primary_adj.OpenTime, primary_adj.OpenPrice, primary_adj.OpenSize, primary_adj.HighTime, primary_adj.HighPrice, primary_adj.LowTime,primary_adj.LowPrice, primary_adj.CloseTime, primary_adj.ClosePrice, primary_adj.CloseSize,primary_adj.ListedMarketHoursVolume, primary_adj.ListedMarketHoursTrades,primary_adj.ListedTotalVolume, primary_adj.ListedTotalTrades, primary_adj.FinraMarketHoursVolume,primary_adj.FinraMarketHoursTrades, primary_adj.FinraTotalVolume, primary_adj.FinraTotalTrades,primary_adj.MarketVWAP, primary_adj.DailyVWAP, primary_adj.PriceAdjFactor, primary_adj.VolumeAdjFactor).filter((primary_adj.Ticker == "ALVR")&(primary_adj.TradeDate > "2022-01-01")).fetch()


In [10]:
data

,TradeDate,SecId,Ticker,Name,PrimaryExchange,ISIN,OpenTime,OpenPrice,OpenSize,HighTime,HighPrice,LowTime,LowPrice,CloseTime,ClosePrice,CloseSize,ListedMarketHoursVolume,ListedMarketHoursTrades,ListedTotalVolume,ListedTotalTrades,FinraMarketHoursVolume,FinraMarketHoursTrades,FinraTotalVolume,FinraTotalTrades,MarketVWAP,DailyVWAP,PriceAdjFactor,VolumeAdjFactor
0,2022-01-03,6466315,ALVR,AlloVir Inc,NASDAQ,US0198181036,2022-01-03 09:30:00.751000-05:00,12.95,2554,2022-01-03 15:59:13.894000-05:00,13.55,2022-01-03 09:44:51.075000-05:00,12.50,2022-01-03 16:00:01.015000-05:00,13.52,22685,115732,2167,115933,2169,44947,728,45825,734,13.32,13.32,1.0,1.0
1,2022-07-12,6466315,ALVR,AlloVir Inc,NASDAQ,US0198181036,2022-07-12 09:30:00.532000-04:00,4.76,1051,2022-07-12 15:56:15.190000-04:00,4.92,2022-07-12 10:12:52.494000-04:00,4.29,2022-07-12 16:00:01.551000-04:00,4.91,7362,129174,2320,129575,2324,68662,596,68962,597,4.62,4.62,1.0,1.0
2,2022-10-12,6466315,ALVR,AlloVir Inc,NASDAQ,US0198181036,2022-10-12 09:30:01.908000-04:00,9.13,1550,2022-10-12 09:30:04.581000-04:00,9.17,2022-10-12 10:23:09.455000-04:00,8.57,2022-10-12 16:00:00.950000-04:00,9.02,14783,130074,2740,130075,2741,78320,598,80681,604,8.89,8.89,1.0,1.0
3,2022-04-06,6466315,ALVR,AlloVir Inc,NASDAQ,US0198181036,2022-04-06 09:30:02.395000-04:00,6.47,1757,2022-04-06 15:02:05.981000-04:00,6.74,2022-04-06 10:29:24.608000-04:00,6.27,2022-04-06 16:00:01.289000-04:00,6.55,19923,131716,2651,131716,2651,42057,881,42515,887,6.51,6.51,1.0,1.0
4,2022-01-04,6466315,ALVR,AlloVir Inc,NASDAQ,US0198181036,2022-01-04 09:30:02.101000-05:00,13.54,1582,2022-01-04 09:30:04.216000-05:00,13.70,2022-01-04 13:26:47.760000-05:00,12.36,2022-01-04 16:00:01.052000-05:00,12.70,28459,139354,2416,139356,2418,61336,925,63316,930,12.73,12.73,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,2022-04-05,6466315,ALVR,AlloVir Inc,NASDAQ,US0198181036,2022-04-05 09:30:01.950000-04:00,7.06,1034,2022-04-05 11:16:24.211000-04:00,7.09,2022-04-05 15:58:42.379000-04:00,6.55,2022-04-05 16:00:01.182000-04:00,6.61,13282,110371,2459,110371,2459,34777,633,36517,641,6.73,6.73,1.0,1.0
256,2022-07-08,6466315,ALVR,AlloVir Inc,NASDAQ,US0198181036,2022-07-08 09:30:01.501000-04:00,4.97,8483,2022-07-08 09:49:05.768000-04:00,5.02,2022-07-08 11:28:28.794000-04:00,4.69,2022-07-08 16:00:01.879000-04:00,4.99,9082,124100,2028,124112,2031,79884,809,80786,813,4.89,4.89,1.0,1.0
257,2022-07-11,6466315,ALVR,AlloVir Inc,NASDAQ,US0198181036,2022-07-11 09:30:00.651000-04:00,4.98,5484,2022-07-11 15:59:46.588000-04:00,4.78,2022-07-11 11:01:51.517000-04:00,4.53,2022-07-11 16:00:00.590000-04:00,4.77,17071,137133,1705,140133,1707,78598,629,78871,632,4.72,4.72,1.0,1.0
258,2022-10-10,6466315,ALVR,AlloVir Inc,NASDAQ,US0198181036,2022-10-10 09:30:01.966000-04:00,8.72,1027,2022-10-10 15:58:59.412000-04:00,9.42,2022-10-10 09:32:44.735000-04:00,8.67,2022-10-10 16:00:00.925000-04:00,9.32,22896,211692,3971,211713,3974,124000,819,133746,838,9.15,9.15,1.0,1.0


In [36]:
data["TradeDate"] = pd.to_datetime(data["TradeDate"])


In [37]:
data["TradeDate"].dt.month

0      10
1       7
2       4
3       1
4       7
       ..
255    10
256     4
257    10
258     4
259     4
Name: TradeDate, Length: 260, dtype: int64

In [30]:
df = pa.Table.from_pandas(data)

In [32]:
df.schema

TradeDate: date32[day]
SecId: int64
Ticker: string
Name: string
PrimaryExchange: string
ISIN: string
OpenTime: timestamp[ns, tz=US/Eastern]
OpenPrice: double
OpenSize: int64
HighTime: timestamp[ns, tz=US/Eastern]
HighPrice: double
LowTime: timestamp[ns, tz=US/Eastern]
LowPrice: double
CloseTime: timestamp[ns, tz=US/Eastern]
ClosePrice: double
CloseSize: int64
ListedMarketHoursVolume: int64
ListedMarketHoursTrades: int64
ListedTotalVolume: int64
ListedTotalTrades: int64
FinraMarketHoursVolume: int64
FinraMarketHoursTrades: int64
FinraTotalVolume: int64
FinraTotalTrades: int64
MarketVWAP: double
DailyVWAP: double
PriceAdjFactor: double
VolumeAdjFactor: double
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 3828

In [ ]:
asyncio.get_event_loop().stop()

In [9]:
import nest_asyncio
nest_asyncio.apply()

In [26]:
import clickhouse_driver
DATABASE_HOST = '54.164.83.119'
DATABASE_USER = "angel_mv_hill_gmail_com"
DATABASE_PASSWORD = "UbxaVAQ572A"
client = clickhouse_driver.Client(
    DATABASE_HOST,
    user=DATABASE_USER,
    password=DATABASE_PASSWORD,
    secure=False
)
client.execute('show databases')

[('USEquityMarketData',), ('USEquityReferenceData',)]

In [33]:
a = client.execute("""SELECT * FROM USEquityMarketData.PrimaryAdjustedOHLCDaily WHERE Ticker = 'AAPL' AND TradeDate = toDate('2023-01-11'); """)
a

[(datetime.date(2023, 1, 11),
  33449,
  'AAPL',
  'Apple Inc',
  'NASDAQ',
  'US0378331005',
  datetime.datetime(2023, 1, 11, 9, 30, 1, 416000, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
  131.25,
  459460,
  datetime.datetime(2023, 1, 11, 15, 56, 0, 90000, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
  133.51,
  datetime.datetime(2023, 1, 11, 9, 41, 59, 340000, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
  130.46,
  datetime.datetime(2023, 1, 11, 16, 0, 0, 843000, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
  133.49,
  7124837,
  33206028,
  305585,
  34431046,
  324130,
  32884764,
  230874,
  35078378,
  237158,
  132.28,
  132.31,
  1.0,
  1.0)]

In [38]:
a1 = client.execute("""SELECT * FROM USEquityMarketData.PrimaryAdjustedOHLCDaily WHERE Ticker = 'AAPL' AND toYear(TradeDate) = 2022; """)
a1

[(datetime.date(2022, 7, 5),
  33449,
  'AAPL',
  'Apple Inc',
  'NASDAQ',
  'US0378331005',
  datetime.datetime(2022, 7, 5, 9, 30, 1, 985000, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  137.2813,
  624965,
  datetime.datetime(2022, 7, 5, 15, 59, 59, 620000, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  141.1794,
  datetime.datetime(2022, 7, 5, 9, 32, 52, 447000, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  136.5136,
  datetime.datetime(2022, 7, 5, 16, 0, 1, 193000, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  141.1295,
  5850140,
  37533423,
  362171,
  38440710,
  377584,
  30831737,
  219000,
  34997887,
  224383,
  139.265211,
  139.344968,
  0.9969590613958513,
  1.0),
 (datetime.date(2022, 1, 3),
  33449,
  'AAPL',
  'Apple Inc',
  'NASDAQ',
  'US0378331005',
  datetime.datetime(2022, 1, 3, 9, 30, 2, 332000, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
  176.784,
  817599,
  datetime.datetime(2022, 1, 3, 13,

In [59]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pandas as pd
from schema import *

In [63]:
list(a1)

[(datetime.date(2022, 7, 5),
  33449,
  'AAPL',
  'Apple Inc',
  'NASDAQ',
  'US0378331005',
  datetime.datetime(2022, 7, 5, 9, 30, 1, 985000, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  137.2813,
  624965,
  datetime.datetime(2022, 7, 5, 15, 59, 59, 620000, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  141.1794,
  datetime.datetime(2022, 7, 5, 9, 32, 52, 447000, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  136.5136,
  datetime.datetime(2022, 7, 5, 16, 0, 1, 193000, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  141.1295,
  5850140,
  37533423,
  362171,
  38440710,
  377584,
  30831737,
  219000,
  34997887,
  224383,
  139.265211,
  139.344968,
  0.9969590613958513,
  1.0),
 (datetime.date(2022, 1, 3),
  33449,
  'AAPL',
  'Apple Inc',
  'NASDAQ',
  'US0378331005',
  datetime.datetime(2022, 1, 3, 9, 30, 2, 332000, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
  176.784,
  817599,
  datetime.datetime(2022, 1, 3, 13,

In [71]:
dfa = pd.DataFrame(a1)

In [70]:
import numpy as np
aa1 = np.array(a1)
aa1[1:]

array([[datetime.date(2022, 1, 3), 33449, 'AAPL', ..., 180.373157,
        0.9942297240049162, 1.0],
       [datetime.date(2022, 10, 3), 33449, 'AAPL', ..., 140.976141,
        0.998343894009217, 1.0],
       [datetime.date(2022, 4, 1), 33449, 'AAPL', ..., 172.629032,
        0.9954964053767088, 1.0],
       ...,
       [datetime.date(2022, 9, 28), 33449, 'AAPL', ..., 147.355559,
        0.998343894009217, 1.0],
       [datetime.date(2022, 9, 29), 33449, 'AAPL', ..., 142.663342,
        0.998343894009217, 1.0],
       [datetime.date(2022, 9, 30), 33449, 'AAPL', ..., 140.147516,
        0.998343894009217, 1.0]], dtype=object)

In [72]:

# py_a = pa.Table.from_arrays(np.array(a1), schema=primary_adj_schema)
py_a = pa.Table.from_pandas(dfa)
py_a

pyarrow.Table
0: date32[day]
1: int64
2: string
3: string
4: string
5: string
6: timestamp[ns, tz=US/Eastern]
7: double
8: int64
9: timestamp[ns, tz=US/Eastern]
10: double
11: timestamp[ns, tz=US/Eastern]
12: double
13: timestamp[ns, tz=US/Eastern]
14: double
15: int64
16: int64
17: int64
18: int64
19: int64
20: int64
21: int64
22: int64
23: int64
24: double
25: double
26: double
27: double
----
0: [[2022-07-05,2022-01-03,2022-10-03,2022-04-01,2022-07-06,...,2022-12-30,2022-09-27,2022-09-28,2022-09-29,2022-09-30]]
1: [[33449,33449,33449,33449,33449,...,33449,33449,33449,33449,33449]]
2: [["AAPL","AAPL","AAPL","AAPL","AAPL",...,"AAPL","AAPL","AAPL","AAPL","AAPL"]]
3: [["Apple Inc","Apple Inc","Apple Inc","Apple Inc","Apple Inc",...,"Apple Inc","Apple Inc","Apple Inc","Apple Inc","Apple Inc"]]
4: [["NASDAQ","NASDAQ","NASDAQ","NASDAQ","NASDAQ",...,"NASDAQ","NASDAQ","NASDAQ","NASDAQ","NASDAQ"]]
5: [["US0378331005","US0378331005","US0378331005","US0378331005","US0378331005",...,"US037833100

In [73]:
%%time
a2 = client.execute("""SELECT * FROM USEquityMarketData.TradeAndQuoteMinuteBar WHERE Ticker = 'AAPL' AND TradeDate = toDate('2023-01-11'); """)
a2

CPU times: total: 0 ns
Wall time: 1.04 s


[(datetime.date(2023, 1, 11),
  datetime.datetime(2023, 1, 11, 4, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
  'AAPL',
  33449,
  Decimal('0'),
  Decimal('0'),
  0,
  Decimal('0'),
  0,
  Decimal('0.00629341'),
  Decimal('130.95'),
  100,
  Decimal('0.042018673'),
  Decimal('131'),
  100,
  Decimal('0.048586585'),
  Decimal('131'),
  200,
  Decimal('0.040824784'),
  Decimal('131.03'),
  585,
  Decimal('0.004351008'),
  Decimal('130.68'),
  400,
  Decimal('51.4154176'),
  Decimal('130.8'),
  100,
  Decimal('52.805085286'),
  Decimal('130.74'),
  34,
  Decimal('59.999999999'),
  Decimal('130.73'),
  200,
  Decimal('130.8'),
  100,
  Decimal('52.805085286'),
  Decimal('130.74'),
  34,
  Decimal('0.01'),
  Decimal('0.27'),
  0,
  Decimal('130.93145'),
  213,
  100,
  179,
  0,
  654,
  2302,
  601,
  3836,
  64,
  0,
  Decimal('0'),
  823,
  837,
  601,
  660,
  915,
  Decimal('8.68131'),
  Decimal('0.36743'),
  Decimal('130.7432'),
  Decimal('130.89634')),
 (datetime.dat

In [ ]:
import threading, queue
import os
import requests
import pandas as pd
import time
from pandas.core.frame import DataFrame

num_workers = 3
records = 50

results = []
failed_requests = []
Ids = set()


faang = ['AAPL','NFLX','AMZN']

queries = []
results = []

def runQuery(query):
    try:
        client = clickhouse_driver.Client(
            DATABASE_HOST,
            user=DATABASE_USER,
            password=DATABASE_PASSWORD,
            secure=False
        )
        result = client.query_dataframe(query)
        results.append(result)
    except:
        print('query failed')

for company in faang:
    query = """ SELECT * 
    FROM USEquityMarketData.TradeAndQuoteMinuteBar
    WHERE Ticker = '%s'
        AND CAST(BarDateTime as Date) >= '2021-01-01'
        AND CAST(BarDateTime as Date) < '2021-01-02'
    Order BY BarDateTime
    """ % (company)
    queries.append(query)
    

In [8]:
queries

[" SELECT * \n    FROM USEquityMarketData.TradeAndQuoteMinuteBar\n    WHERE Ticker = 'AAPL'\n        AND CAST(BarDateTime as Date) >= '2021-01-01'\n        AND CAST(BarDateTime as Date) < '2021-01-02'\n    Order BY BarDateTime\n    ",
 " SELECT * \n    FROM USEquityMarketData.TradeAndQuoteMinuteBar\n    WHERE Ticker = 'NFLX'\n        AND CAST(BarDateTime as Date) >= '2021-01-01'\n        AND CAST(BarDateTime as Date) < '2021-01-02'\n    Order BY BarDateTime\n    ",
 " SELECT * \n    FROM USEquityMarketData.TradeAndQuoteMinuteBar\n    WHERE Ticker = 'AMZN'\n        AND CAST(BarDateTime as Date) >= '2021-01-01'\n        AND CAST(BarDateTime as Date) < '2021-01-02'\n    Order BY BarDateTime\n    "]

In [22]:
daily = ds.dataset('data/us_equity/example_daily',partitioning='hive',format='parquet')
daily

In [23]:
daily.schema

TradeDate: timestamp[us]
Ticker: string
Name: string
PrimaryExchange: string
ISIN: string
OpenTime: timestamp[us, tz=US/Eastern]
OpenPrice: double
OpenSize: int64
HighTime: timestamp[us, tz=US/Eastern]
HighPrice: double
LowTime: timestamp[us, tz=US/Eastern]
LowPrice: double
CloseTime: timestamp[us, tz=US/Eastern]
ClosePrice: double
CloseSize: int64
ListedMarketHoursVolume: int64
ListedMarketHoursTrades: int64
ListedTotalVolume: int64
ListedTotalTrades: int64
FinraMarketHoursVolume: int64
FinraMarketHoursTrades: int64
FinraTotalVolume: int64
FinraTotalTrades: int64
MarketVWAP: double
DailyVWAP: double
PriceAdjFactor: double
VolumeAdjFactor: double
Year: int32
SecId: int32
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 3945

In [25]:
daily.count_rows()

820871